In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('../input/unetresnet34trainedmodel/unet-resnet34.h5', compile=False)

In [ ]:
#https://www.kaggle.com/titericz/building-and-visualizing-masks
#https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode

#defining function for converting EncodedPixels(rle: run length encoding) to mask
def rle2mask(rle_string, img_shape=(256,1600)):
    '''
    input: EncodedPixels (run-length-encoded) string & image shape:-(width,height)
    output: mask in numpy.ndarray format with shape (256,1600)
    '''
    rle_array = np.array([int(s)for s in rle_string.split()])
    starts_array = rle_array[::2]-1
    lengths_array = rle_array[1::2]
    mask_array = np.zeros(img_shape[0]*img_shape[1],dtype=np.uint8)
    #print(starts_array,lengths_array)
    for i in range(len(starts_array)):
        mask_array[starts_array[i]:starts_array[i]+lengths_array[i]] = 1
    #order='F' because encoded pixels are numbered from top to bottom, then left to right
    return mask_array.reshape(img_shape, order = 'F')

#defining function for converting given mask to EncodedPixels(rle: run length encoding)
def mask2rle(mask_array):
    '''
    input: mask in numpy.ndarray format
    output: EncodedPixels (run-length-encoded) string
    '''
    mask_array = mask_array.T.flatten()
    mask_array = np.concatenate([[0], mask_array, [0]])
    rle_array = np.where(mask_array[1:]!=mask_array[:-1])[0]+1
    rle_array[1::2] -= rle_array[::2]
    rle_string = ' '.join(map(str,rle_array))
    return rle_string

#defining function for calculation of metric dice coefficient
def dice_coefficient(y_true, y_pred):
  y_true_f = tf.reshape(y_true, [-1])
  y_pred_f = tf.reshape(y_pred, [-1])
  intersection = tf.math.reduce_sum(y_true_f * y_pred_f)
  smoothing_const = 1e-9
  return (2. * intersection + smoothing_const) / (tf.math.reduce_sum(y_true_f) + tf.math.reduce_sum(y_pred_f) + smoothing_const)

In [ ]:
class PredictDataGenerator(tf.keras.utils.Sequence):
    def __init__(self,dataframe, list_idcs, batch_size=32, ):
        self.batch_size = batch_size
        self.df = dataframe
        self.list_idcs = list_idcs
        self.indices = self.df.index.tolist()
        self.rem = len(self.list_idcs) % (self.batch_size)
        self.on_epoch_end()

    def __len__(self):
         return len(self.list_idcs) // (self.batch_size)
#         if (self.rem) == 0:
#             return len(self.list_idcs) // (self.batch_size)
#         else:
#             return (len(self.list_idcs) // (self.batch_size) )+1

    def __getitem__(self, index):
        index = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
#         if ((index + 1) * self.batch_size) < len(self.list_idcs):
#             index = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
#         else:
#             index = self.indices[index * self.batch_size: (index * self.batch_size)+ self.rem]
        batch = [self.list_idcs[k] for k in index]
        
        X = self.__get_data(batch)
         
        return X
    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))

    def __get_data(self, batch):
        X = np.empty((self.batch_size,256,1600,3),dtype=np.float32) # image place-holders
              
        for i, id in enumerate(batch):
          img = Image.open('../input/severstal-steel-defect-detection/test_images/' + str(self.df['ImageId'].loc[id]))
          X[i,] = img#input image

        return X

In [ ]:
test_img_IDs = list(os.listdir('../input/severstal-steel-defect-detection/test_images'))
test_imgsIds_df = pd.DataFrame({'ImageId': test_img_IDs})
print(len(test_imgsIds_df))
test_imgsIds_df.head()

In [ ]:
SubmissionDf = pd.DataFrame(columns = ['ImageId','EncodedPixels','ClassId'])

In [ ]:
for i in range(0,len(test_imgsIds_df),320):
    batch_idcs =  list(range(i, min(test_imgsIds_df.shape[0], i + 320)))#.iloc[batch_idcs]
    if len(batch_idcs)== 320:        
        test_subbatch = PredictDataGenerator(dataframe = test_imgsIds_df,
                                             list_idcs = batch_idcs)
    else:
        test_subbatch = PredictDataGenerator(dataframe = test_imgsIds_df,
                                             list_idcs = batch_idcs,
                                             batch_size= len(batch_idcs))
    #print(len(test_subbatch))
    subbatch_pred_masks = model.predict(test_subbatch)
    #print(len(subbatch_pred_masks))
    #break
    for j, idx in tqdm(enumerate(batch_idcs)):
        filename = test_imgsIds_df['ImageId'].iloc[idx]
        rle1 = mask2rle(subbatch_pred_masks[j,:,:,0].round().astype(int))
        rle2 = mask2rle(subbatch_pred_masks[j,:,:,1].round().astype(int))
        rle3 = mask2rle(subbatch_pred_masks[j,:,:,2].round().astype(int))
        rle4 = mask2rle(subbatch_pred_masks[j,:,:,3].round().astype(int))
        df = pd.DataFrame({'ImageId':[filename]*4,
                      'EncodedPixels': [rle1,rle2,rle3,rle4],
                      'ClassId':['1', '2', '3', '4']})
        SubmissionDf = SubmissionDf.append(df,ignore_index=True)
        #print(SubmissionDf.head())
        #print(SubmissionDf.shape)
        #break
    #break

In [ ]:
SubmissionDf.sort_values(by=['ImageId', 'ClassId'], inplace=True)
print(SubmissionDf.shape)
SubmissionDf.head(10)

In [ ]:
SubmissionDf['ImageId_ClassId'] = SubmissionDf['ImageId'] + '_' + SubmissionDf['ClassId']
SubmissionDf

In [ ]:
SubmissionDf[['ImageId_ClassId','EncodedPixels' ]].to_csv('submission.csv', index=False)